# <center> CNN, what's Fresh? </center> 
## <center> for Text Classification & Sentiment Analysis </center> 


### Using Convolutional Neural Network for sentiment analysis of movie reviews

In this tutorial we will implement a Convolutional Neural Network, a neural network architecture, which has been commonly utilized for image classification problems, to classify movie reviews from http://rottentomoatoes.com. These movie reviews, either from critics or public audience, are either **"Fresh"**, conveying a **_positive overall review_**, or **"Rotten"** for **_negative_** ones. We will train & evaluate a CNN model to classify 1-sentence movie reviews from Rottentomatoes.com.

![Cont. Convolution](./images/incredibles_review.png)

### 1. Dataset

The dataset consists of a bit over 10,000, one-sentence, movie reviews from the Rotten Tomatoes web site in 2005. Each review represents an opinion of the movie as either "Fresh" or "Rotten. The dataset can be downoaded directly downloaded from Cornell, here: http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz

### 2. Convolutional Neural Networks

CNN is a multi layered neural network architecture. Several CNN variants (e.g. LeNet, AlexNet) have yielded very good results in image classification problems. Advances in GPU computing in the last decade made it much easier and faster to train CNNs on large image datasets. Although the current accuracy world records for image classification (e.g. the [ImageNet ILSVRC challenge](http://www.image-net.org/challenges/LSVRC/)) are held by much more complicated and deeper networks, e.g. the 152-layer ResNet, which was the first network to surpass human accuracy level (2015), and Inception v3 (2016), convolution is still at the core of these advanced  networks.

### What is convolution?

Convolution is a mathematical operation, combining two functions (signals) to generate a new signal. Historically, convolution has been an essential tool in DSP (Digital Signal Processing), e.g. an impulse response signal can be convolved with base signal to render a certain effect. For example, in audio processing, impulse response samples have been utilized to add the hallmark 'reverb' effect of famous concert halls to studio recordings, see [Wiki article](https://en.wikipedia.org/wiki/Convolution_reverb). 

The image below illustrates how a pluse (square) signal (in red) is convolved with the base signal (blue), producing the new signal (black).

![Cont. Convolution](./images/Convolution_of_spiky_function_with_box2.gif)

In DSP, convolution (as illustrated above) is usually performed on continuous signals. In image processing, convolution of discrete signals, represented by matrices, yields a class of filters that allow us to manipulate images in several different ways. **Convolution filters could be configured to sharpen, blur, or even detect edges in a given image.** A greyscale image can be represented as a large matrix, one matrix entry per pixel, a kernel matrix (much smaller matrix), can be convolved with original matrix (image) as illustrated below. Colored images can be processed as separate channels (RGB), so a colored image is represented by 3 (greyscale) matrices.

Take a look at the following figures and notice the dimension reduction due to the convolution process.
![convolution](./images/RiverTrain-ImageConvDiagram.png)
<img src="./images/Convolution_schematic.gif" width="400">

Example of a blurring effect
<img src="./images/gimp_blur.png" width="400">

Example of edge-detection
<img src="./images/gimp_edge.png" width="400">

### How do convolution networks work for image classification?

One major challenge for classical classification algorithms when applied to image datasets is the inherent high-dimensionality of the data. If each row of the dataset is a 300x300 image, this actually yields a flattened 90,000-dimensional dataset! 

CNNs, on the other hand, utilize several convolution & pooling (e.g. Max pooling, a moving window extracting pixel of highest value) layers for both dimension reduction & feature selection processes. The "weights" of the convolution kernels (along with weights for activation functions, e.g. ReLU) are iteratively optimized in the process. Optimization is performed to minimize a certain "loss" function, whose optimal minimal value corresponds to a high classification accuracy. "Stochastic Gradient Descent" is a common algorithm for the minimization of the loss function. The weights are updated each iteration based on a back-probagation scheme (mathematically = chain rule) to update the gradient based on the error in classification in previous iteration. 

<img src="./images/cnn.png" width="700">
<img src="./images/cnn.jpg" width="630">

### How would they work for text classification!?

We can think of each word (token) of a sentence as a row in a matrix. Technically, each row is rather a numeric vector representation of the given token, usually via a word embedding algorithm like Word2Vec, GLoVe, or even a simple one-hot encoding to index each word with respect to the corpus of words/tokens in our dataset. So, for example, a 10-word sentence (with vector embedding in a 100 dimensional space) can be represented by a 10X100 matrix (image!).

The convolution filters in this case, have the same width, say a 100 per prior example, and height may vary depending on the use-case, covering 2-5 word span is typical.

### But still, sentences are not very similar to images!

Well, that is in fact true, adjacent words in a sentence (per grammar & syntax rules) are not really similar to adjacent pixels in an image. Besides, certain intuitive image transformations do not yield meaningful results for language applications. So how should we expect CNNs to yield good results? 

Shouldn't we use better suited architectures (e.g. RNNs, LSTMs, Sequence-to-sequence models) for NLP applications? That is also true, but from practical experience, CNNs still deliver good accuracy for simple language tasks, and the real gain here is that they are much easier and faster to train compared to other more advanced architectures. For more complicated language tasks, e.g. **100,000+ Questions for Machine Comprehension of Text**, more specialized & elaborate architectures may be required, see [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) for more on the latest developments.

<img src="./images/cnn_text1.png" width="700">
<img src="./images/cnn_text2.png" width="700">

### 3. The Network

For this particular use-case the network used is based on the one introduced in the paper ["Convolutional Neural Networks for Sentence Classification"](https://arxiv.org/abs/1408.5882) (see above image) with minor changes, as follows: 

* Paper utilizes 2 channels for the input layer (see image above), one is static (constant through training), and a dynamic layer that is fine-tuned via back probagation. We'll have only one static channel.

* We will not utilize a pre-built embedding model (e.g. Word2vec or GLoVe), but instead, an embedding layer will be implemented to "learn" the embedding during the training of the model.

* We pad the sentences to have all the same length, 59 words/tokens per sentence per our dataset.

* Filter sizes, i.e. the number of words we want our convolutional filters to cover. For example, [3, 4, 5] means that we will have filters that slide over 3, 4 and 5 words respectively, for a total of 3 * num_filters filters.

#### Regularization

Neural network models tend to overfit over the training data, so a regularization mechanism will help in this regard. Standard regularization techniques (e.g. L1 (LASSO), L2 (ridge)) have been found not to be very effective for these models, see this [paper](https://arxiv.org/abs/1510.03820) for more detailed information. Instead, the "Dropout" technique is used. The idea behind the dropout technique is to randomly disable a number of neurons in the network, so as to prevent too many neurons from co-adapting, and hence force the neurons to individiually learn more relevant features.

### 4. Training code

#### Training the model
To train the model on the processed dataset, we will need to excute the `train.py` script in Python (ver 3.x).

http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/

In [ ]:
%run -i './train_tf_1_8_updated.py' # runs/1530522078 --> embed=128_filters=128_dropout=0.5


Parameters:

Loading data...
Vocabulary Size: 18758
Train/Dev split: 9596/1066
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity inste

2018-07-02T04:01:33.969893: step 99, loss 1.54095, acc 0.5
2018-07-02T04:01:34.123934: step 100, loss 1.08209, acc 0.640625

Evaluation:
2018-07-02T04:01:34.589521: step 100, loss 0.794583, acc 0.557223

Saved model checkpoint to /Users/jostine.ho/Desktop/cnn4text/runs/1530522078/checkpoints/model-100

2018-07-02T04:01:34.900468: step 101, loss 1.0267, acc 0.625
2018-07-02T04:01:35.049111: step 102, loss 1.61225, acc 0.53125
2018-07-02T04:01:35.197327: step 103, loss 1.53591, acc 0.578125
2018-07-02T04:01:35.353156: step 104, loss 1.20092, acc 0.546875
2018-07-02T04:01:35.506930: step 105, loss 1.49103, acc 0.5
2018-07-02T04:01:35.654408: step 106, loss 1.58463, acc 0.5
2018-07-02T04:01:35.800607: step 107, loss 1.65028, acc 0.53125
2018-07-02T04:01:35.950469: step 108, loss 1.28861, acc 0.59375
2018-07-02T04:01:36.118958: step 109, loss 1.39751, acc 0.515625
2018-07-02T04:01:36.285642: step 110, loss 1.51959, acc 0.53125
2018-07-02T04:01:36.443599: step 111, loss 1.76743, acc 0.515625

2018-07-02T04:01:53.641714: step 223, loss 0.851062, acc 0.640625
2018-07-02T04:01:53.783086: step 224, loss 0.730356, acc 0.734375
2018-07-02T04:01:53.928983: step 225, loss 0.996203, acc 0.578125
2018-07-02T04:01:54.068896: step 226, loss 0.668391, acc 0.640625
2018-07-02T04:01:54.212793: step 227, loss 0.95424, acc 0.609375
2018-07-02T04:01:54.353357: step 228, loss 1.07132, acc 0.5625
2018-07-02T04:01:54.504496: step 229, loss 0.820201, acc 0.640625
2018-07-02T04:01:54.648827: step 230, loss 0.735556, acc 0.65625
2018-07-02T04:01:54.793336: step 231, loss 1.01152, acc 0.5
2018-07-02T04:01:54.934694: step 232, loss 0.951127, acc 0.71875
2018-07-02T04:01:55.082449: step 233, loss 1.02178, acc 0.640625
2018-07-02T04:01:55.218371: step 234, loss 0.883643, acc 0.625
2018-07-02T04:01:55.364685: step 235, loss 1.00158, acc 0.5625
2018-07-02T04:01:55.502137: step 236, loss 1.00421, acc 0.625
2018-07-02T04:01:55.652274: step 237, loss 0.755697, acc 0.5625
2018-07-02T04:01:55.812249: step 23

2018-07-02T04:02:13.196270: step 347, loss 0.718369, acc 0.625
2018-07-02T04:02:13.338206: step 348, loss 0.638937, acc 0.65625
2018-07-02T04:02:13.487286: step 349, loss 0.757143, acc 0.640625
2018-07-02T04:02:13.646478: step 350, loss 0.690385, acc 0.65625
2018-07-02T04:02:13.806756: step 351, loss 0.617644, acc 0.65625
2018-07-02T04:02:13.964849: step 352, loss 0.686999, acc 0.71875
2018-07-02T04:02:14.119313: step 353, loss 0.462394, acc 0.796875
2018-07-02T04:02:14.266080: step 354, loss 0.696542, acc 0.625
2018-07-02T04:02:14.408264: step 355, loss 0.346339, acc 0.828125
2018-07-02T04:02:14.555886: step 356, loss 0.608413, acc 0.71875
2018-07-02T04:02:14.714275: step 357, loss 0.755848, acc 0.625
2018-07-02T04:02:14.863319: step 358, loss 0.692391, acc 0.671875
2018-07-02T04:02:15.015940: step 359, loss 0.61255, acc 0.671875
2018-07-02T04:02:15.182417: step 360, loss 0.536202, acc 0.703125
2018-07-02T04:02:15.334075: step 361, loss 0.649882, acc 0.65625
2018-07-02T04:02:15.494763

2018-07-02T04:02:32.534293: step 471, loss 0.574905, acc 0.734375
2018-07-02T04:02:32.676177: step 472, loss 0.669197, acc 0.640625
2018-07-02T04:02:32.830120: step 473, loss 0.808157, acc 0.515625
2018-07-02T04:02:32.983308: step 474, loss 0.676046, acc 0.59375
2018-07-02T04:02:33.132723: step 475, loss 0.561214, acc 0.703125
2018-07-02T04:02:33.280136: step 476, loss 0.743896, acc 0.6875
2018-07-02T04:02:33.428511: step 477, loss 0.582039, acc 0.734375
2018-07-02T04:02:33.576529: step 478, loss 0.601675, acc 0.671875
2018-07-02T04:02:33.724036: step 479, loss 0.534855, acc 0.78125
2018-07-02T04:02:33.867072: step 480, loss 0.594735, acc 0.6875
2018-07-02T04:02:34.008957: step 481, loss 0.508201, acc 0.71875
2018-07-02T04:02:34.150606: step 482, loss 0.598458, acc 0.640625
2018-07-02T04:02:34.297204: step 483, loss 0.423955, acc 0.796875
2018-07-02T04:02:34.452347: step 484, loss 0.580242, acc 0.734375
2018-07-02T04:02:34.606220: step 485, loss 0.683801, acc 0.65625
2018-07-02T04:02:3

...

2018-07-02T04:10:13.103379: step 3437, loss 0.00640349, acc 1
2018-07-02T04:10:13.239552: step 3438, loss 0.0204142, acc 1
2018-07-02T04:10:13.375054: step 3439, loss 0.00239557, acc 1
2018-07-02T04:10:13.513436: step 3440, loss 0.00899236, acc 1
2018-07-02T04:10:13.655679: step 3441, loss 0.0317875, acc 0.984375
2018-07-02T04:10:13.800223: step 3442, loss 0.00900918, acc 1
2018-07-02T04:10:13.942578: step 3443, loss 0.0344958, acc 0.984375
2018-07-02T04:10:14.080138: step 3444, loss 0.0336015, acc 0.984375
2018-07-02T04:10:14.222802: step 3445, loss 0.0112514, acc 1
2018-07-02T04:10:14.365343: step 3446, loss 0.00310805, acc 1
2018-07-02T04:10:14.514092: step 3447, loss 0.00855861, acc 1
2018-07-02T04:10:14.656481: step 3448, loss 0.007081, acc 1
2018-07-02T04:10:14.804152: step 3449, loss 0.00253422, acc 1
2018-07-02T04:10:14.938701: step 3450, loss 0.00435468, acc 1
2018-07-02T04:10:15.084994: step 3451, loss 0.0153832, acc 1
2018-07-02T04:10:15.230170: step 3452, loss 0.00476405, a

2018-07-02T04:10:32.528671: step 3565, loss 0.00553709, acc 1
2018-07-02T04:10:32.669565: step 3566, loss 0.0051066, acc 1
2018-07-02T04:10:32.810734: step 3567, loss 0.00332169, acc 1
2018-07-02T04:10:32.952565: step 3568, loss 0.00438771, acc 1
2018-07-02T04:10:33.092559: step 3569, loss 0.00300396, acc 1
2018-07-02T04:10:33.231548: step 3570, loss 0.00568565, acc 1
2018-07-02T04:10:33.378122: step 3571, loss 0.00769622, acc 1
2018-07-02T04:10:33.524051: step 3572, loss 0.00302253, acc 1
2018-07-02T04:10:33.667505: step 3573, loss 0.00298692, acc 1
2018-07-02T04:10:33.808077: step 3574, loss 0.0143003, acc 1
2018-07-02T04:10:33.950082: step 3575, loss 0.0160136, acc 1
2018-07-02T04:10:34.111345: step 3576, loss 0.013393, acc 1
2018-07-02T04:10:34.270284: step 3577, loss 0.00591628, acc 1
2018-07-02T04:10:34.413689: step 3578, loss 0.00678685, acc 1
2018-07-02T04:10:34.556158: step 3579, loss 0.0285036, acc 1
2018-07-02T04:10:34.698503: step 3580, loss 0.00882588, acc 1
2018-07-02T04:

2018-07-02T04:10:52.649337: step 3695, loss 0.00732774, acc 1
2018-07-02T04:10:52.791261: step 3696, loss 0.00975733, acc 1
2018-07-02T04:10:52.933371: step 3697, loss 0.00296161, acc 1
2018-07-02T04:10:53.074472: step 3698, loss 0.0108849, acc 1
2018-07-02T04:10:53.216029: step 3699, loss 0.002611, acc 1
2018-07-02T04:10:53.358506: step 3700, loss 0.0140696, acc 1

Evaluation:
2018-07-02T04:10:53.821512: step 3700, loss 1.19037, acc 0.739212

Saved model checkpoint to /Users/jostine.ho/Desktop/cnn4text/runs/1530522078/checkpoints/model-3700

2018-07-02T04:10:54.135104: step 3701, loss 0.00366938, acc 1
2018-07-02T04:10:54.279216: step 3702, loss 0.00795939, acc 1
2018-07-02T04:10:54.448254: step 3703, loss 0.00237613, acc 1
2018-07-02T04:10:54.607584: step 3704, loss 0.00443928, acc 1
2018-07-02T04:10:54.765192: step 3705, loss 0.0020878, acc 1
2018-07-02T04:10:54.912251: step 3706, loss 0.00185893, acc 1
2018-07-02T04:10:55.057423: step 3707, loss 0.00206694, acc 1
2018-07-02T04:10:5

2018-07-02T04:11:12.633734: step 3821, loss 0.00360763, acc 1
2018-07-02T04:11:12.781198: step 3822, loss 0.00328785, acc 1
2018-07-02T04:11:12.924171: step 3823, loss 0.00136134, acc 1
2018-07-02T04:11:13.074121: step 3824, loss 0.00418039, acc 1
2018-07-02T04:11:13.220257: step 3825, loss 0.00618391, acc 1
2018-07-02T04:11:13.389496: step 3826, loss 0.00175868, acc 1
2018-07-02T04:11:13.552683: step 3827, loss 0.00365493, acc 1
2018-07-02T04:11:13.704898: step 3828, loss 0.000254924, acc 1
2018-07-02T04:11:13.852576: step 3829, loss 0.0129996, acc 1
2018-07-02T04:11:13.992478: step 3830, loss 0.0209221, acc 0.984375
2018-07-02T04:11:14.135536: step 3831, loss 0.00199895, acc 1
2018-07-02T04:11:14.299526: step 3832, loss 0.0056053, acc 1
2018-07-02T04:11:14.448929: step 3833, loss 0.00359342, acc 1
2018-07-02T04:11:14.589919: step 3834, loss 0.000537191, acc 1
2018-07-02T04:11:14.732992: step 3835, loss 0.0016733, acc 1
2018-07-02T04:11:14.867859: step 3836, loss 0.00909074, acc 1
201

2018-07-02T04:11:32.943869: step 3951, loss 0.00103263, acc 1
2018-07-02T04:11:33.102711: step 3952, loss 0.00334063, acc 1
2018-07-02T04:11:33.250469: step 3953, loss 0.000694871, acc 1
2018-07-02T04:11:33.391459: step 3954, loss 0.00140062, acc 1
2018-07-02T04:11:33.544113: step 3955, loss 0.00206971, acc 1
2018-07-02T04:11:33.686954: step 3956, loss 0.0156937, acc 1
2018-07-02T04:11:33.831909: step 3957, loss 0.00089499, acc 1
2018-07-02T04:11:33.973523: step 3958, loss 0.00396011, acc 1
2018-07-02T04:11:34.121152: step 3959, loss 0.0092569, acc 1
2018-07-02T04:11:34.270279: step 3960, loss 0.0028524, acc 1
2018-07-02T04:11:34.428789: step 3961, loss 0.00247052, acc 1
2018-07-02T04:11:34.573068: step 3962, loss 0.00492343, acc 1
2018-07-02T04:11:34.729635: step 3963, loss 0.00854382, acc 1
2018-07-02T04:11:34.870018: step 3964, loss 0.00250936, acc 1
2018-07-02T04:11:35.016775: step 3965, loss 0.00518381, acc 1
2018-07-02T04:11:35.155023: step 3966, loss 0.00489624, acc 1
2018-07-02

In [1]:
%run -i './train_tf_1_8_updated.py'


Parameters:

Loading data...
Vocabulary Size: 18758
Train/Dev split: 9596/1066
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity inste

2018-07-02T12:34:52.329107: step 97, loss 1.94566, acc 0.59375
2018-07-02T12:34:52.690789: step 98, loss 2.21493, acc 0.53125
2018-07-02T12:34:53.131420: step 99, loss 2.21971, acc 0.5
2018-07-02T12:34:53.578373: step 100, loss 2.06292, acc 0.515625

Evaluation:
2018-07-02T12:34:55.010191: step 100, loss 1.10932, acc 0.573171

Saved model checkpoint to /Users/jostine.ho/Desktop/cnn4text/runs/1530552851/checkpoints/model-100

2018-07-02T12:34:55.665959: step 101, loss 1.65892, acc 0.546875
2018-07-02T12:34:56.065324: step 102, loss 1.67489, acc 0.5625
2018-07-02T12:34:56.475541: step 103, loss 1.84099, acc 0.59375
2018-07-02T12:34:56.836181: step 104, loss 1.60504, acc 0.515625
2018-07-02T12:34:57.220426: step 105, loss 1.64482, acc 0.625
2018-07-02T12:34:57.559770: step 106, loss 1.3389, acc 0.609375
2018-07-02T12:34:57.895887: step 107, loss 1.61233, acc 0.546875
2018-07-02T12:34:58.222681: step 108, loss 1.53262, acc 0.5
2018-07-02T12:34:58.606108: step 109, loss 2.01141, acc 0.5
201

2018-07-02T12:35:41.606248: step 220, loss 0.950951, acc 0.65625
2018-07-02T12:35:42.066948: step 221, loss 1.14182, acc 0.625
2018-07-02T12:35:42.427816: step 222, loss 1.09897, acc 0.59375
2018-07-02T12:35:42.824561: step 223, loss 1.37086, acc 0.640625
2018-07-02T12:35:43.256077: step 224, loss 1.04468, acc 0.71875
2018-07-02T12:35:43.695670: step 225, loss 1.17737, acc 0.59375
2018-07-02T12:35:44.102814: step 226, loss 1.35812, acc 0.59375
2018-07-02T12:35:44.489504: step 227, loss 1.27704, acc 0.6875
2018-07-02T12:35:44.973033: step 228, loss 0.980409, acc 0.640625
2018-07-02T12:35:45.347098: step 229, loss 1.05489, acc 0.671875
2018-07-02T12:35:45.675289: step 230, loss 0.783943, acc 0.71875
2018-07-02T12:35:46.036917: step 231, loss 1.13224, acc 0.640625
2018-07-02T12:35:46.363859: step 232, loss 1.11247, acc 0.546875
2018-07-02T12:35:46.701197: step 233, loss 0.859294, acc 0.65625
2018-07-02T12:35:46.996744: step 234, loss 0.891558, acc 0.625
2018-07-02T12:35:47.341179: step 23

2018-07-02T12:36:29.781354: step 344, loss 0.880425, acc 0.71875
2018-07-02T12:36:30.082649: step 345, loss 0.598708, acc 0.765625
2018-07-02T12:36:30.390432: step 346, loss 0.618983, acc 0.75
2018-07-02T12:36:30.687570: step 347, loss 0.696645, acc 0.734375
2018-07-02T12:36:31.008194: step 348, loss 0.708098, acc 0.703125
2018-07-02T12:36:31.322227: step 349, loss 0.632432, acc 0.703125
2018-07-02T12:36:31.635795: step 350, loss 0.638503, acc 0.703125
2018-07-02T12:36:31.953079: step 351, loss 0.536186, acc 0.75
2018-07-02T12:36:32.369345: step 352, loss 0.749218, acc 0.6875
2018-07-02T12:36:32.733795: step 353, loss 0.752079, acc 0.6875
2018-07-02T12:36:33.093727: step 354, loss 0.567242, acc 0.71875
2018-07-02T12:36:33.472561: step 355, loss 0.631656, acc 0.75
2018-07-02T12:36:33.944333: step 356, loss 0.636062, acc 0.78125
2018-07-02T12:36:34.355632: step 357, loss 0.574897, acc 0.734375
2018-07-02T12:36:34.780382: step 358, loss 0.59157, acc 0.71875
2018-07-02T12:36:35.236187: ste

2018-07-02T12:37:16.601419: step 468, loss 0.614029, acc 0.703125
2018-07-02T12:37:16.907611: step 469, loss 0.528854, acc 0.734375
2018-07-02T12:37:17.232256: step 470, loss 0.414732, acc 0.78125
2018-07-02T12:37:17.556138: step 471, loss 0.702251, acc 0.640625
2018-07-02T12:37:17.887595: step 472, loss 0.572428, acc 0.71875
2018-07-02T12:37:18.214015: step 473, loss 0.649931, acc 0.6875
2018-07-02T12:37:18.544446: step 474, loss 0.416715, acc 0.796875
2018-07-02T12:37:18.879290: step 475, loss 0.61217, acc 0.75
2018-07-02T12:37:19.182949: step 476, loss 0.478031, acc 0.78125
2018-07-02T12:37:19.491514: step 477, loss 0.62039, acc 0.71875
2018-07-02T12:37:19.814725: step 478, loss 0.340582, acc 0.8125
2018-07-02T12:37:20.140237: step 479, loss 0.521995, acc 0.8125
2018-07-02T12:37:20.461765: step 480, loss 0.522485, acc 0.671875
2018-07-02T12:37:20.784511: step 481, loss 0.642634, acc 0.71875
2018-07-02T12:37:21.154967: step 482, loss 0.677577, acc 0.671875
2018-07-02T12:37:21.469548:

KeyboardInterrupt: 

In [1]:
%run -i './train_tf_1_8_updated.py' # runs/1530523402 --> embed=128_filters=64_dropout=0.5


Parameters:

Loading data...
Vocabulary Size: 18758
Train/Dev split: 9596/1066
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity inste

2018-07-02T04:23:35.561399: step 99, loss 1.12035, acc 0.546875
2018-07-02T04:23:35.682775: step 100, loss 1.69937, acc 0.453125

Evaluation:
2018-07-02T04:23:36.064495: step 100, loss 0.748395, acc 0.548781

Saved model checkpoint to /Users/jostine.ho/Desktop/cnn4text/runs/1530523402/checkpoints/model-100

2018-07-02T04:23:36.339753: step 101, loss 0.786396, acc 0.625
2018-07-02T04:23:36.473704: step 102, loss 1.16445, acc 0.546875
2018-07-02T04:23:36.601604: step 103, loss 1.36482, acc 0.578125
2018-07-02T04:23:36.739315: step 104, loss 1.15395, acc 0.53125
2018-07-02T04:23:36.871663: step 105, loss 1.17216, acc 0.640625
2018-07-02T04:23:37.001558: step 106, loss 1.43181, acc 0.46875
2018-07-02T04:23:37.134041: step 107, loss 1.32989, acc 0.453125
2018-07-02T04:23:37.270044: step 108, loss 1.09652, acc 0.546875
2018-07-02T04:23:37.398351: step 109, loss 1.09817, acc 0.578125
2018-07-02T04:23:37.528645: step 110, loss 1.45267, acc 0.4375
2018-07-02T04:23:37.657035: step 111, loss 1.10

2018-07-02T04:23:52.586728: step 223, loss 0.867654, acc 0.5625
2018-07-02T04:23:52.711411: step 224, loss 0.799276, acc 0.71875
2018-07-02T04:23:52.846549: step 225, loss 1.12269, acc 0.453125
2018-07-02T04:23:52.966206: step 226, loss 0.838771, acc 0.546875
2018-07-02T04:23:53.092235: step 227, loss 0.715028, acc 0.671875
2018-07-02T04:23:53.212038: step 228, loss 0.754644, acc 0.609375
2018-07-02T04:23:53.339923: step 229, loss 0.951813, acc 0.546875
2018-07-02T04:23:53.475274: step 230, loss 0.878374, acc 0.515625
2018-07-02T04:23:53.607885: step 231, loss 0.830252, acc 0.5
2018-07-02T04:23:53.739270: step 232, loss 0.812124, acc 0.5625
2018-07-02T04:23:53.877779: step 233, loss 0.842835, acc 0.53125
2018-07-02T04:23:54.017417: step 234, loss 0.690507, acc 0.546875
2018-07-02T04:23:54.153323: step 235, loss 0.760404, acc 0.578125
2018-07-02T04:23:54.293293: step 236, loss 0.742566, acc 0.65625
2018-07-02T04:23:54.429611: step 237, loss 0.840906, acc 0.578125
2018-07-02T04:23:54.550

2018-07-02T04:24:08.826695: step 347, loss 0.619277, acc 0.6875
2018-07-02T04:24:08.975215: step 348, loss 0.71524, acc 0.546875
2018-07-02T04:24:09.112882: step 349, loss 0.579905, acc 0.640625
2018-07-02T04:24:09.236356: step 350, loss 0.727572, acc 0.609375
2018-07-02T04:24:09.362698: step 351, loss 0.644081, acc 0.640625
2018-07-02T04:24:09.487401: step 352, loss 0.576745, acc 0.75
2018-07-02T04:24:09.614800: step 353, loss 0.65403, acc 0.59375
2018-07-02T04:24:09.740034: step 354, loss 0.671602, acc 0.625
2018-07-02T04:24:09.860754: step 355, loss 0.618288, acc 0.703125
2018-07-02T04:24:09.982706: step 356, loss 0.560561, acc 0.6875
2018-07-02T04:24:10.101386: step 357, loss 0.58833, acc 0.640625
2018-07-02T04:24:10.227028: step 358, loss 0.681534, acc 0.546875
2018-07-02T04:24:10.349365: step 359, loss 0.668743, acc 0.578125
2018-07-02T04:24:10.473468: step 360, loss 0.698163, acc 0.609375
2018-07-02T04:24:10.595641: step 361, loss 0.748619, acc 0.578125
2018-07-02T04:24:10.71967

2018-07-02T04:24:24.876020: step 471, loss 0.554577, acc 0.65625
2018-07-02T04:24:25.007279: step 472, loss 0.664264, acc 0.625
2018-07-02T04:24:25.130035: step 473, loss 0.62662, acc 0.671875
2018-07-02T04:24:25.252097: step 474, loss 0.637468, acc 0.609375
2018-07-02T04:24:25.369650: step 475, loss 0.59019, acc 0.671875
2018-07-02T04:24:25.492695: step 476, loss 0.61486, acc 0.671875
2018-07-02T04:24:25.627559: step 477, loss 0.608015, acc 0.6875
2018-07-02T04:24:25.771395: step 478, loss 0.583067, acc 0.65625
2018-07-02T04:24:25.902952: step 479, loss 0.542855, acc 0.734375
2018-07-02T04:24:26.025759: step 480, loss 0.654905, acc 0.6875
2018-07-02T04:24:26.146905: step 481, loss 0.550687, acc 0.734375
2018-07-02T04:24:26.269782: step 482, loss 0.673109, acc 0.640625
2018-07-02T04:24:26.397819: step 483, loss 0.581417, acc 0.71875
2018-07-02T04:24:26.528980: step 484, loss 0.577539, acc 0.65625
2018-07-02T04:24:26.655324: step 485, loss 0.641822, acc 0.609375
2018-07-02T04:24:26.7825

...

2018-07-02T05:04:29.492862: step 18741, loss 2.55181e-07, acc 1
2018-07-02T05:04:29.612074: step 18742, loss 3.40971e-05, acc 1
2018-07-02T05:04:29.747908: step 18743, loss 0.000395612, acc 1
2018-07-02T05:04:29.883389: step 18744, loss 5.80161e-05, acc 1
2018-07-02T05:04:30.006383: step 18745, loss 2.42129e-06, acc 1
2018-07-02T05:04:30.133060: step 18746, loss 0.000170188, acc 1
2018-07-02T05:04:30.255937: step 18747, loss 2.08423e-06, acc 1
2018-07-02T05:04:30.376294: step 18748, loss 0.000145571, acc 1
2018-07-02T05:04:30.500211: step 18749, loss 8.68724e-05, acc 1
2018-07-02T05:04:30.621994: step 18750, loss 5.5193e-05, acc 1
2018-07-02T05:04:30.742753: step 18751, loss 1.77877e-06, acc 1
2018-07-02T05:04:30.869133: step 18752, loss 7.87326e-05, acc 1
2018-07-02T05:04:31.000610: step 18753, loss 6.16503e-06, acc 1
2018-07-02T05:04:31.129897: step 18754, loss 1.88126e-07, acc 1
2018-07-02T05:04:31.257664: step 18755, loss 9.27528e-05, acc 1
2018-07-02T05:04:31.399668: step 18756, l

2018-07-02T05:04:46.146293: step 18867, loss 9.3132e-08, acc 1
2018-07-02T05:04:46.274969: step 18868, loss 2.62217e-05, acc 1
2018-07-02T05:04:46.409950: step 18869, loss 0.14442, acc 0.984375
2018-07-02T05:04:46.540806: step 18870, loss 2.4269e-06, acc 1
2018-07-02T05:04:46.686348: step 18871, loss 1.32435e-05, acc 1
2018-07-02T05:04:46.818698: step 18872, loss 0.000260695, acc 1
2018-07-02T05:04:46.955870: step 18873, loss 0.0452915, acc 0.984375
2018-07-02T05:04:47.083103: step 18874, loss 3.01177e-06, acc 1
2018-07-02T05:04:47.208317: step 18875, loss 2.01695e-05, acc 1
2018-07-02T05:04:47.340523: step 18876, loss 4.36916e-06, acc 1
2018-07-02T05:04:47.469411: step 18877, loss 1.01886e-06, acc 1
2018-07-02T05:04:47.598851: step 18878, loss 0.0076464, acc 1
2018-07-02T05:04:47.737194: step 18879, loss 4.51069e-06, acc 1
2018-07-02T05:04:47.877721: step 18880, loss 1.69692e-05, acc 1
2018-07-02T05:04:48.011714: step 18881, loss 9.92541e-06, acc 1
2018-07-02T05:04:48.133864: step 188

2018-07-02T05:05:02.616563: step 18993, loss 0.000294166, acc 1
2018-07-02T05:05:02.741219: step 18994, loss 1.04293e-05, acc 1
2018-07-02T05:05:02.873228: step 18995, loss 1.13246e-06, acc 1
2018-07-02T05:05:02.997381: step 18996, loss 6.54842e-06, acc 1
2018-07-02T05:05:03.118515: step 18997, loss 7.08844e-06, acc 1
2018-07-02T05:05:03.239715: step 18998, loss 1.83002e-05, acc 1
2018-07-02T05:05:03.358572: step 18999, loss 5.95433e-05, acc 1
2018-07-02T05:05:03.485951: step 19000, loss 6.87235e-06, acc 1

Evaluation:
2018-07-02T05:05:03.773462: step 19000, loss 5.12615, acc 0.719512

Saved model checkpoint to /Users/jostine.ho/Desktop/cnn4text/runs/1530523402/checkpoints/model-19000

2018-07-02T05:05:04.023603: step 19001, loss 2.28921e-05, acc 1
2018-07-02T05:05:04.146884: step 19002, loss 1.49012e-08, acc 1
2018-07-02T05:05:04.270193: step 19003, loss 3.01607e-05, acc 1
2018-07-02T05:05:04.407435: step 19004, loss 0.000105228, acc 1
2018-07-02T05:05:04.540824: step 19005, loss 1.00

2018-07-02T05:05:19.286410: step 19117, loss 0.000818813, acc 1
2018-07-02T05:05:19.407603: step 19118, loss 1.53777e-05, acc 1
2018-07-02T05:05:19.545825: step 19119, loss 0.00070339, acc 1
2018-07-02T05:05:19.683687: step 19120, loss 0.000414833, acc 1
2018-07-02T05:05:19.813929: step 19121, loss 0.000395146, acc 1
2018-07-02T05:05:19.936843: step 19122, loss 0.000569546, acc 1
2018-07-02T05:05:20.050290: step 19123, loss 1.41398e-05, acc 1
2018-07-02T05:05:20.170327: step 19124, loss 1.14827e-05, acc 1
2018-07-02T05:05:20.288677: step 19125, loss 0.000129851, acc 1
2018-07-02T05:05:20.401632: step 19126, loss 4.47034e-08, acc 1
2018-07-02T05:05:20.517152: step 19127, loss 0.000778353, acc 1
2018-07-02T05:05:20.645723: step 19128, loss 0.000361039, acc 1
2018-07-02T05:05:20.789917: step 19129, loss 1.69242e-05, acc 1
2018-07-02T05:05:20.926031: step 19130, loss 2.6132e-06, acc 1
2018-07-02T05:05:21.060358: step 19131, loss 0.000233367, acc 1
2018-07-02T05:05:21.190037: step 19132, lo

KeyboardInterrupt: 

In [1]:
%run -i './train_tf_1_8_updated.py' # runs/1530527810 --> embed=128_filters=128_dropout=0.2


Parameters:

Loading data...
Vocabulary Size: 18758
Train/Dev split: 9596/1066
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity inste

2018-07-02T05:37:05.926789: step 99, loss 2.35993, acc 0.46875
2018-07-02T05:37:06.063364: step 100, loss 2.19336, acc 0.515625

Evaluation:
2018-07-02T05:37:06.668013: step 100, loss 0.736766, acc 0.582552

Saved model checkpoint to /Users/jostine.ho/Desktop/cnn4text/runs/1530527810/checkpoints/model-100

2018-07-02T05:37:06.959183: step 101, loss 2.11031, acc 0.609375
2018-07-02T05:37:07.105536: step 102, loss 3.05262, acc 0.5
2018-07-02T05:37:07.267801: step 103, loss 2.48573, acc 0.40625
2018-07-02T05:37:07.412555: step 104, loss 3.56231, acc 0.296875
2018-07-02T05:37:07.565831: step 105, loss 2.84731, acc 0.5
2018-07-02T05:37:07.717835: step 106, loss 2.42729, acc 0.46875
2018-07-02T05:37:07.859728: step 107, loss 2.06911, acc 0.5625
2018-07-02T05:37:07.993750: step 108, loss 2.24268, acc 0.578125
2018-07-02T05:37:08.133451: step 109, loss 1.87037, acc 0.578125
2018-07-02T05:37:08.272271: step 110, loss 2.49117, acc 0.46875
2018-07-02T05:37:08.408738: step 111, loss 1.68986, acc 0

2018-07-02T05:37:25.477323: step 223, loss 1.26214, acc 0.5
2018-07-02T05:37:25.622066: step 224, loss 1.09812, acc 0.578125
2018-07-02T05:37:25.772259: step 225, loss 1.29752, acc 0.46875
2018-07-02T05:37:25.925164: step 226, loss 1.06858, acc 0.59375
2018-07-02T05:37:26.094562: step 227, loss 1.02593, acc 0.609375
2018-07-02T05:37:26.264984: step 228, loss 1.03358, acc 0.46875
2018-07-02T05:37:26.428183: step 229, loss 1.27649, acc 0.515625
2018-07-02T05:37:26.579581: step 230, loss 1.03305, acc 0.546875
2018-07-02T05:37:26.727587: step 231, loss 1.40194, acc 0.40625
2018-07-02T05:37:26.870244: step 232, loss 1.17239, acc 0.5625
2018-07-02T05:37:27.016842: step 233, loss 1.10318, acc 0.625
2018-07-02T05:37:27.154969: step 234, loss 0.903556, acc 0.5625
2018-07-02T05:37:27.303625: step 235, loss 1.12959, acc 0.546875
2018-07-02T05:37:27.465332: step 236, loss 1.45927, acc 0.484375
2018-07-02T05:37:27.642560: step 237, loss 0.852965, acc 0.65625
2018-07-02T05:37:27.805740: step 238, lo

2018-07-02T05:37:44.874175: step 349, loss 0.77824, acc 0.578125
2018-07-02T05:37:45.011460: step 350, loss 0.792009, acc 0.5625
2018-07-02T05:37:45.148300: step 351, loss 0.769107, acc 0.546875
2018-07-02T05:37:45.281782: step 352, loss 0.710628, acc 0.5625
2018-07-02T05:37:45.442549: step 353, loss 0.839536, acc 0.59375
2018-07-02T05:37:45.597614: step 354, loss 0.715106, acc 0.546875
2018-07-02T05:37:45.735982: step 355, loss 0.673533, acc 0.578125
2018-07-02T05:37:45.870694: step 356, loss 0.816705, acc 0.5
2018-07-02T05:37:46.006556: step 357, loss 0.710561, acc 0.609375
2018-07-02T05:37:46.149323: step 358, loss 0.725005, acc 0.625
2018-07-02T05:37:46.291664: step 359, loss 0.797206, acc 0.546875
2018-07-02T05:37:46.430867: step 360, loss 0.763218, acc 0.5
2018-07-02T05:37:46.573392: step 361, loss 0.739723, acc 0.59375
2018-07-02T05:37:46.714753: step 362, loss 0.911377, acc 0.515625
2018-07-02T05:37:46.855897: step 363, loss 0.748389, acc 0.578125
2018-07-02T05:37:47.007596: st

2018-07-02T05:38:03.677753: step 473, loss 0.758213, acc 0.546875
2018-07-02T05:38:03.820123: step 474, loss 0.69996, acc 0.546875
2018-07-02T05:38:03.961156: step 475, loss 0.609963, acc 0.625
2018-07-02T05:38:04.099606: step 476, loss 0.729413, acc 0.546875
2018-07-02T05:38:04.237269: step 477, loss 0.777486, acc 0.515625
2018-07-02T05:38:04.377349: step 478, loss 0.757345, acc 0.515625
2018-07-02T05:38:04.514572: step 479, loss 0.775914, acc 0.578125
2018-07-02T05:38:04.655338: step 480, loss 0.727995, acc 0.5625
2018-07-02T05:38:04.796313: step 481, loss 0.658583, acc 0.671875
2018-07-02T05:38:04.935402: step 482, loss 0.813056, acc 0.46875
2018-07-02T05:38:05.075902: step 483, loss 0.700966, acc 0.5625
2018-07-02T05:38:05.226147: step 484, loss 0.688776, acc 0.640625
2018-07-02T05:38:05.387756: step 485, loss 0.741924, acc 0.546875
2018-07-02T05:38:05.526903: step 486, loss 0.748097, acc 0.5625
2018-07-02T05:38:05.663732: step 487, loss 0.690225, acc 0.578125
2018-07-02T05:38:05.8

...

2018-07-02T05:48:52.337590: step 4651, loss 0.00324161, acc 1
2018-07-02T05:48:52.478792: step 4652, loss 0.0607503, acc 0.953125
2018-07-02T05:48:52.617925: step 4653, loss 0.0046195, acc 1
2018-07-02T05:48:52.754023: step 4654, loss 0.00904491, acc 1
2018-07-02T05:48:52.894594: step 4655, loss 0.0100304, acc 1
2018-07-02T05:48:53.041868: step 4656, loss 0.0299243, acc 0.984375
2018-07-02T05:48:53.182837: step 4657, loss 0.00661897, acc 1
2018-07-02T05:48:53.322455: step 4658, loss 0.00522061, acc 1
2018-07-02T05:48:53.461601: step 4659, loss 0.0102033, acc 1
2018-07-02T05:48:53.597041: step 4660, loss 0.00723507, acc 1
2018-07-02T05:48:53.735813: step 4661, loss 0.0356931, acc 0.984375
2018-07-02T05:48:53.894023: step 4662, loss 0.0261665, acc 0.984375
2018-07-02T05:48:54.048337: step 4663, loss 0.0121259, acc 1
2018-07-02T05:48:54.188225: step 4664, loss 0.024182, acc 0.984375
2018-07-02T05:48:54.324422: step 4665, loss 0.0175731, acc 1
2018-07-02T05:48:54.462283: step 4666, loss 0.

2018-07-02T05:49:10.922649: step 4779, loss 0.00504528, acc 1
2018-07-02T05:49:11.061054: step 4780, loss 0.0445248, acc 0.984375
2018-07-02T05:49:11.200052: step 4781, loss 0.124525, acc 0.984375
2018-07-02T05:49:11.336903: step 4782, loss 0.0219464, acc 0.984375
2018-07-02T05:49:11.476226: step 4783, loss 0.0189204, acc 0.984375
2018-07-02T05:49:11.614913: step 4784, loss 0.00257365, acc 1
2018-07-02T05:49:11.754279: step 4785, loss 0.00616165, acc 1
2018-07-02T05:49:11.891787: step 4786, loss 0.121166, acc 0.953125
2018-07-02T05:49:12.030263: step 4787, loss 0.0162904, acc 1
2018-07-02T05:49:12.168198: step 4788, loss 0.00385998, acc 1
2018-07-02T05:49:12.307104: step 4789, loss 0.0212203, acc 0.984375
2018-07-02T05:49:12.443230: step 4790, loss 0.0055273, acc 1
2018-07-02T05:49:12.582248: step 4791, loss 0.00475955, acc 1
2018-07-02T05:49:12.720380: step 4792, loss 0.0442114, acc 0.984375
2018-07-02T05:49:12.859373: step 4793, loss 0.00445139, acc 1
2018-07-02T05:49:12.999190: step

2018-07-02T05:49:29.602173: step 4903, loss 0.0168511, acc 1
2018-07-02T05:49:29.752246: step 4904, loss 0.00510468, acc 1
2018-07-02T05:49:29.908657: step 4905, loss 0.000971817, acc 1
2018-07-02T05:49:30.047460: step 4906, loss 0.00833165, acc 1
2018-07-02T05:49:30.197721: step 4907, loss 0.0220143, acc 0.984375
2018-07-02T05:49:30.345363: step 4908, loss 0.0390583, acc 0.984375
2018-07-02T05:49:30.504890: step 4909, loss 0.0181135, acc 1
2018-07-02T05:49:30.649990: step 4910, loss 0.00370858, acc 1
2018-07-02T05:49:30.790150: step 4911, loss 0.00466793, acc 1
2018-07-02T05:49:30.928939: step 4912, loss 0.0140321, acc 1
2018-07-02T05:49:31.064282: step 4913, loss 0.0041064, acc 1
2018-07-02T05:49:31.205658: step 4914, loss 0.051457, acc 0.984375
2018-07-02T05:49:31.348579: step 4915, loss 0.00224878, acc 1
2018-07-02T05:49:31.492563: step 4916, loss 0.00711477, acc 1
2018-07-02T05:49:31.635614: step 4917, loss 0.0179242, acc 0.984375
2018-07-02T05:49:31.787464: step 4918, loss 0.0144

2018-07-02T05:49:49.186023: step 5031, loss 0.00092255, acc 1
2018-07-02T05:49:49.333310: step 5032, loss 0.0066154, acc 1
2018-07-02T05:49:49.474113: step 5033, loss 0.00576972, acc 1
2018-07-02T05:49:49.615420: step 5034, loss 0.0174368, acc 0.984375
2018-07-02T05:49:49.761909: step 5035, loss 0.0180416, acc 1
2018-07-02T05:49:49.898532: step 5036, loss 0.0200815, acc 0.984375
2018-07-02T05:49:50.056846: step 5037, loss 0.0264277, acc 0.984375
2018-07-02T05:49:50.215176: step 5038, loss 0.00815414, acc 1
2018-07-02T05:49:50.359049: step 5039, loss 0.0238974, acc 0.984375
2018-07-02T05:49:50.501012: step 5040, loss 0.0545259, acc 0.984375
2018-07-02T05:49:50.643919: step 5041, loss 0.0107257, acc 1
2018-07-02T05:49:50.782165: step 5042, loss 0.012019, acc 1
2018-07-02T05:49:50.922091: step 5043, loss 0.00929151, acc 1
2018-07-02T05:49:51.064290: step 5044, loss 0.0167395, acc 1
2018-07-02T05:49:51.200350: step 5045, loss 0.00602579, acc 1
2018-07-02T05:49:51.337639: step 5046, loss 0.

KeyboardInterrupt: 

In [1]:
%run -i './train_tf_1_8_updated.py' # runs/1530528908 --> embed=256_filters=128_dropout=0.5


Parameters:

Loading data...
Vocabulary Size: 18758
Train/Dev split: 9596/1066
INFO:tensorflow:Summary name embedding/W:0/grad/hist is illegal; using embedding/W_0/grad/hist instead.
INFO:tensorflow:Summary name embedding/W:0/grad/sparsity is illegal; using embedding/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/hist is illegal; using conv-maxpool-3/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0/grad/sparsity is illegal; using conv-maxpool-3/W_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/hist is illegal; using conv-maxpool-3/b_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0/grad/sparsity is illegal; using conv-maxpool-3/b_0/grad/sparsity instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/hist is illegal; using conv-maxpool-4/W_0/grad/hist instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0/grad/sparsity is illegal; using conv-maxpool-4/W_0/grad/sparsity inste

2018-07-02T05:55:38.416038: step 98, loss 1.9585, acc 0.59375
2018-07-02T05:55:38.704303: step 99, loss 1.91925, acc 0.59375
2018-07-02T05:55:39.025095: step 100, loss 2.21096, acc 0.484375

Evaluation:
2018-07-02T05:55:40.004232: step 100, loss 1.24299, acc 0.542214

Saved model checkpoint to /Users/jostine.ho/Desktop/cnn4text/runs/1530528908/checkpoints/model-100

2018-07-02T05:55:40.539723: step 101, loss 2.14063, acc 0.484375
2018-07-02T05:55:40.844268: step 102, loss 1.43622, acc 0.578125
2018-07-02T05:55:41.164136: step 103, loss 1.74121, acc 0.546875
2018-07-02T05:55:41.476052: step 104, loss 1.90515, acc 0.5625
2018-07-02T05:55:41.790568: step 105, loss 2.56559, acc 0.4375
2018-07-02T05:55:42.102115: step 106, loss 1.79989, acc 0.59375
2018-07-02T05:55:42.390267: step 107, loss 1.4626, acc 0.484375
2018-07-02T05:55:42.681974: step 108, loss 1.47089, acc 0.53125
2018-07-02T05:55:42.968948: step 109, loss 2.26122, acc 0.484375
2018-07-02T05:55:43.254309: step 110, loss 1.5487, ac

2018-07-02T05:56:17.864918: step 221, loss 0.960029, acc 0.640625
2018-07-02T05:56:18.198030: step 222, loss 1.08706, acc 0.625
2018-07-02T05:56:18.481123: step 223, loss 1.10643, acc 0.59375
2018-07-02T05:56:18.759560: step 224, loss 0.725405, acc 0.703125
2018-07-02T05:56:19.036624: step 225, loss 1.25706, acc 0.53125
2018-07-02T05:56:19.316260: step 226, loss 0.646804, acc 0.609375
2018-07-02T05:56:19.604632: step 227, loss 0.919389, acc 0.703125
2018-07-02T05:56:19.906742: step 228, loss 0.819561, acc 0.65625
2018-07-02T05:56:20.211052: step 229, loss 1.20179, acc 0.625
2018-07-02T05:56:20.504030: step 230, loss 0.905991, acc 0.65625
2018-07-02T05:56:20.804883: step 231, loss 1.19382, acc 0.609375
2018-07-02T05:56:21.104818: step 232, loss 0.868815, acc 0.6875
2018-07-02T05:56:21.426983: step 233, loss 0.982197, acc 0.640625
2018-07-02T05:56:21.723954: step 234, loss 1.03878, acc 0.625
2018-07-02T05:56:22.026043: step 235, loss 0.974555, acc 0.640625
2018-07-02T05:56:22.325805: ste

2018-07-02T05:56:57.911598: step 345, loss 0.487516, acc 0.796875
2018-07-02T05:56:58.217527: step 346, loss 0.812453, acc 0.625
2018-07-02T05:56:58.523966: step 347, loss 0.441979, acc 0.765625
2018-07-02T05:56:58.816755: step 348, loss 0.746817, acc 0.703125
2018-07-02T05:56:59.117029: step 349, loss 0.543731, acc 0.75
2018-07-02T05:56:59.436125: step 350, loss 0.661801, acc 0.6875
2018-07-02T05:56:59.736644: step 351, loss 0.582135, acc 0.671875
2018-07-02T05:57:00.085366: step 352, loss 0.587264, acc 0.703125
2018-07-02T05:57:00.437132: step 353, loss 0.468673, acc 0.796875
2018-07-02T05:57:00.745295: step 354, loss 0.636999, acc 0.703125
2018-07-02T05:57:01.045941: step 355, loss 0.542946, acc 0.75
2018-07-02T05:57:01.360021: step 356, loss 0.744859, acc 0.65625
2018-07-02T05:57:01.686206: step 357, loss 0.698248, acc 0.671875
2018-07-02T05:57:02.047837: step 358, loss 0.553376, acc 0.703125
2018-07-02T05:57:02.382033: step 359, loss 0.856927, acc 0.625
2018-07-02T05:57:02.690922:

2018-07-02T05:57:35.978141: step 469, loss 0.743465, acc 0.671875
2018-07-02T05:57:36.263788: step 470, loss 0.58433, acc 0.75
2018-07-02T05:57:36.570930: step 471, loss 0.5942, acc 0.6875
2018-07-02T05:57:36.860885: step 472, loss 0.447698, acc 0.78125
2018-07-02T05:57:37.153794: step 473, loss 0.456881, acc 0.78125
2018-07-02T05:57:37.437309: step 474, loss 0.504381, acc 0.78125
2018-07-02T05:57:37.724948: step 475, loss 0.539827, acc 0.703125
2018-07-02T05:57:38.011568: step 476, loss 0.489826, acc 0.765625
2018-07-02T05:57:38.295475: step 477, loss 0.419928, acc 0.84375
2018-07-02T05:57:38.583048: step 478, loss 0.499649, acc 0.71875
2018-07-02T05:57:38.870180: step 479, loss 0.620473, acc 0.71875
2018-07-02T05:57:39.162696: step 480, loss 0.408021, acc 0.8125
2018-07-02T05:57:39.476150: step 481, loss 0.55551, acc 0.65625
2018-07-02T05:57:39.756325: step 482, loss 0.502037, acc 0.84375
2018-07-02T05:57:40.036830: step 483, loss 0.449606, acc 0.78125
2018-07-02T05:57:40.314479: ste

...

2018-07-02T06:16:09.383242: step 4074, loss 0.00251802, acc 1
2018-07-02T06:16:09.667070: step 4075, loss 0.000539819, acc 1
2018-07-02T06:16:09.948996: step 4076, loss 0.00349107, acc 1
2018-07-02T06:16:10.232345: step 4077, loss 0.00134871, acc 1
2018-07-02T06:16:10.586289: step 4078, loss 0.00197139, acc 1
2018-07-02T06:16:10.868234: step 4079, loss 0.000446406, acc 1
2018-07-02T06:16:11.152472: step 4080, loss 0.00590744, acc 1
2018-07-02T06:16:11.450670: step 4081, loss 0.00492706, acc 1
2018-07-02T06:16:11.777973: step 4082, loss 0.00189941, acc 1
2018-07-02T06:16:12.094055: step 4083, loss 0.00178926, acc 1
2018-07-02T06:16:12.385022: step 4084, loss 0.00749593, acc 1
2018-07-02T06:16:12.673071: step 4085, loss 0.0036026, acc 1
2018-07-02T06:16:12.963261: step 4086, loss 0.031474, acc 0.984375
2018-07-02T06:16:13.250262: step 4087, loss 0.000340786, acc 1
2018-07-02T06:16:13.533480: step 4088, loss 0.0100228, acc 1
2018-07-02T06:16:13.815392: step 4089, loss 0.00546991, acc 1
20

2018-07-02T06:16:48.967182: step 4201, loss 0.000884591, acc 1
2018-07-02T06:16:49.257568: step 4202, loss 0.00059681, acc 1
2018-07-02T06:16:49.552382: step 4203, loss 0.000563504, acc 1
2018-07-02T06:16:49.823551: step 4204, loss 0.00458086, acc 1
2018-07-02T06:16:50.109674: step 4205, loss 0.0163559, acc 0.984375
2018-07-02T06:16:50.407422: step 4206, loss 0.00299715, acc 1
2018-07-02T06:16:50.759976: step 4207, loss 0.000921065, acc 1
2018-07-02T06:16:51.068343: step 4208, loss 0.00206108, acc 1
2018-07-02T06:16:51.362194: step 4209, loss 0.00102862, acc 1
2018-07-02T06:16:51.698941: step 4210, loss 0.00163118, acc 1
2018-07-02T06:16:51.986703: step 4211, loss 0.00160607, acc 1
2018-07-02T06:16:52.275202: step 4212, loss 0.0019254, acc 1
2018-07-02T06:16:52.563406: step 4213, loss 0.00690256, acc 1
2018-07-02T06:16:52.845797: step 4214, loss 0.0228898, acc 0.96875
2018-07-02T06:16:53.127242: step 4215, loss 0.0013915, acc 1
2018-07-02T06:16:53.413349: step 4216, loss 0.000767575, a

2018-07-02T06:17:27.028824: step 4329, loss 0.0305708, acc 0.984375
2018-07-02T06:17:27.300072: step 4330, loss 0.018221, acc 0.984375
2018-07-02T06:17:27.568089: step 4331, loss 0.00211495, acc 1
2018-07-02T06:17:27.837417: step 4332, loss 0.00885155, acc 1
2018-07-02T06:17:28.114095: step 4333, loss 0.000616368, acc 1
2018-07-02T06:17:28.394950: step 4334, loss 0.00085613, acc 1
2018-07-02T06:17:28.682046: step 4335, loss 0.00226335, acc 1
2018-07-02T06:17:28.989492: step 4336, loss 0.0244494, acc 0.984375
2018-07-02T06:17:29.344000: step 4337, loss 0.0206031, acc 0.984375
2018-07-02T06:17:29.627379: step 4338, loss 0.00291487, acc 1
2018-07-02T06:17:29.922670: step 4339, loss 0.000921186, acc 1
2018-07-02T06:17:30.207354: step 4340, loss 0.000667049, acc 1
2018-07-02T06:17:30.496736: step 4341, loss 0.0463416, acc 0.96875
2018-07-02T06:17:30.791941: step 4342, loss 0.00140226, acc 1
2018-07-02T06:17:31.119651: step 4343, loss 0.000371223, acc 1
2018-07-02T06:17:31.412738: step 4344,

2018-07-02T06:18:05.463901: step 4457, loss 0.00289367, acc 1
2018-07-02T06:18:05.752804: step 4458, loss 0.00230277, acc 1
2018-07-02T06:18:06.022466: step 4459, loss 0.000997119, acc 1
2018-07-02T06:18:06.297063: step 4460, loss 0.000386454, acc 1
2018-07-02T06:18:06.573597: step 4461, loss 0.00166118, acc 1
2018-07-02T06:18:06.851641: step 4462, loss 0.00783504, acc 1
2018-07-02T06:18:07.123565: step 4463, loss 0.00652584, acc 1
2018-07-02T06:18:07.415886: step 4464, loss 0.00296352, acc 1
2018-07-02T06:18:07.697365: step 4465, loss 0.0151868, acc 0.984375
2018-07-02T06:18:07.978547: step 4466, loss 0.00138863, acc 1
2018-07-02T06:18:08.267499: step 4467, loss 0.00163645, acc 1
2018-07-02T06:18:08.557709: step 4468, loss 0.00221679, acc 1
2018-07-02T06:18:08.836171: step 4469, loss 0.00260423, acc 1
2018-07-02T06:18:09.104424: step 4470, loss 0.0135366, acc 1
2018-07-02T06:18:09.373452: step 4471, loss 0.00498713, acc 1
2018-07-02T06:18:09.645445: step 4472, loss 0.0020666, acc 1
20

KeyboardInterrupt: 

### 5. Visualize model using TensorBoard

![Cont. Convolution](./images/tensorboard.png)

In [ ]:
%%bash tensorboard --logdir /Users/jostine.ho/Desktop/cnn4text/runs/

### 6. Evaluation code

In [4]:
# Function to evaluate movie reviews
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from text_cnn import TextCNN
from tensorflow.contrib import learn
path = "/Users/jostine.ho/Desktop/cnn4text/runs/1530522078" 
        ## Make sure to change to your local path
    
def evaluate(x_raw, checkpoint_dir= path + "/checkpoints"):
    batch_size = 64
    vocab_path = os.path.join(path, "vocab")
    vocab_processor = learn.preprocessing.VocabularyProcessor.restore(vocab_path)
    x_test = np.array(list(vocab_processor.transform(x_raw)))

    print("\nEvaluating...\n")
    print(checkpoint_dir)
    checkpoint_file = tf.train.latest_checkpoint(checkpoint_dir)
    
    graph = tf.Graph()
    with graph.as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=True,
          log_device_placement=False)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            # Load the saved meta graph and restore variables
            saver = tf.train.import_meta_graph("{}.meta".format(checkpoint_file))
            saver.restore(sess, checkpoint_file)

            # Get the placeholders from the graph by name
            input_x = graph.get_operation_by_name("input_x").outputs[0]
            # input_y = graph.get_operation_by_name("input_y").outputs[0]
            dropout_keep_prob = graph.get_operation_by_name("dropout_keep_prob").outputs[0]

            # Tensors we want to evaluate
            predictions = graph.get_operation_by_name("output/predictions").outputs[0]

            # Generate batches for one epoch
            batches = data_helpers.batch_iter(list(x_test), batch_size, 1, shuffle=False)

            # Collect the predictions here
            all_predictions = []

            for x_test_batch in batches:
                batch_predictions = sess.run(predictions, {input_x: x_test_batch, dropout_keep_prob: 1.0})
                all_predictions = np.concatenate([all_predictions, batch_predictions])
    print(all_predictions)

![Cont. Convolution](./images/review_pos_1.png)
![Cont. Convolution](./images/review_neg_1.png)
![Cont. Convolution](./images/review_pos_2.png)
![Cont. Convolution](./images/review_neg_2.png)

In [11]:
incredibles_reviews = [
    "Pixar's award-winning quality standards in terms of story and animation remain intact, and this is a very good quality sequel, with a very similar storyline to the first film.",
    "regresses to a time when any power women managed to acquire was carefully controlled so as not constitute a threat to the male order. Such nostalgia is self-defeating.",
    "may not be superior to the original, but it was well worth the wait.",
    "Usually when watching a comic book movie, I walk out of the theater desperate to live in their world. I would hate to run into The Incredibles in real life because I'd be certain they would be looking at me with judgement in their eyes"
                      ]

In [12]:
# embed=128_filters=128_dropout=0.5
path = "/Users/jostine.ho/Desktop/cnn4text/runs/1530522078"
evaluate(incredibles_reviews, checkpoint_dir= path + "/checkpoints")


Evaluating...

/Users/jostine.ho/Desktop/cnn4text/runs/1530522078/checkpoints
[1. 0. 1. 1.]


In [13]:
# embed=128_filters=64_dropout=0.5
path = "/Users/jostine.ho/Desktop/cnn4text/runs/1530523402"
evaluate(incredibles_reviews, checkpoint_dir= path + "/checkpoints")


Evaluating...

/Users/jostine.ho/Desktop/cnn4text/runs/1530523402/checkpoints
[1. 0. 1. 0.]


In [14]:
# embed=128_filters=128_dropout=0.2
path = "/Users/jostine.ho/Desktop/cnn4text/runs/1530527810"
evaluate(incredibles_reviews, checkpoint_dir= path + "/checkpoints")


Evaluating...

/Users/jostine.ho/Desktop/cnn4text/runs/1530527810/checkpoints
[1. 1. 1. 0.]


In [15]:
# embed=256_filters=128_dropout=0.5
path = "/Users/jostine.ho/Desktop/cnn4text/runs/1530528908"
evaluate(incredibles_reviews, checkpoint_dir= path + "/checkpoints")


Evaluating...

/Users/jostine.ho/Desktop/cnn4text/runs/1530528908/checkpoints
[1. 1. 1. 1.]


In [10]:
# Reviews for "IT (2017)"
# embed=128_filters=64_dropout=0.5
path = "/Users/jostine.ho/Desktop/cnn4text/runs/1530523402"
evaluate(["Doesn't cut very deep and isn't very scary", 
          "Creepy and yet emotionally resonant, It is a disturbing reboot of a scary '80s film character",
          "A spooky horror film and a stellar coming of age drama",
          "It only seems like fun if you fall for King's obvious, contemptuous treatment of American innocence",
          "I didn't get it, mediocre at best"], 
          checkpoint_dir= path + "/checkpoints")


Evaluating...

/Users/jostine.ho/Desktop/cnn4text/runs/1530523402/checkpoints
[1. 1. 1. 0. 0.]


#### References

1. Code source, tutorial, more details http://www.wildml.com/2015/12/implementing-a-cnn-for-text-classification-in-tensorflow/
2. https://github.com/dennybritz/cnn-text-classification-tf
3. [Convolutional Neural Networks for Sentence Classification (arxiv)](https://arxiv.org/abs/1408.5882)
4. http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/
5. CNN Sentiment Analysis presentation/demo for Hartford Insurance Group at Stevens Inst of Tech, by Mo Badaway on September 25th, 2017.


#### Image Sources:
Source: http://colah.github.io/posts/2014-07-Understanding-Convolutions/
